In [7]:
# set up and dependencies
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.types import Date
from sqlalchemy.orm import Session
from sqlalchemy import func
from sqlalchemy.sql.expression import extract
import pandas as pd 
from pprint import pprint

%reload_ext nb_black

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
# set up Base

Base = declarative_base()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
#create class for measurement table

class Measurement(Base):
    __tablename__ = "measurement"
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
#create class for station table

class Station(Base):
    __tablename__ = "station"
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation =  Column(Float)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
# create engine and session to link to the database

engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()
session = Session(bind=engine) # If something breaks, check here <-

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Temperature Analysis I

In [13]:
# get the average temperature in June at all stations in all years

avg_june_temp = session.query(func.round(func.avg(Measurement.tobs)))\
    .filter(extract("month", Measurement.date) == "6").first()[0]

avg_june_temp

75.0

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
# get the average temperature in December at all stations in all years

avg_dec_temp = session.query(func.round(func.avg(Measurement.tobs)))\
    .filter(extract("month", Measurement.date) == "12").first()[0]

avg_dec_temp

71.0

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>